In [7]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col
import pandas as pd
import numpy as np
import pickle
import pyarrow.parquet

In [8]:
%%time
targets = pd.read_parquet("hdfs:///projetos/covariate_shift/bureau/targets_hashed.parquet", engine="pyarrow", use_threads=True) #pd.read_parquet("hdfs:///projetos/covariate_shift/bureau/len_src_data.parquet")
targets.shape

CPU times: user 5.36 s, sys: 3.43 s, total: 8.79 s
Wall time: 8.62 s


(10000000, 12)

In [3]:
targets=targets.loc[:,['DT_T0','NDOC','SConcEver30dP3_100']]
targets=targets.rename(columns={"DT_T0": "dt_t0", "NDOC": "ndoc"})

In [4]:
%%time

real_data = pd.read_parquet("hdfs:///projetos/covariate_shift/bureau/book_positivo_hashed.parquet", engine="pyarrow", use_threads=True) 
real_data.shape

CPU times: user 33min 51s, sys: 16min 11s, total: 50min 3s
Wall time: 16min 19s


(10000001, 1794)

In [5]:
dates_feat=['2019-08-16','2019-09-15','2019-10-15','2019-11-14',
            '2019-12-14','2020-01-13','2020-02-12','2020-03-13','2020-04-12','2020-05-12']
dates_targ=['2019-08-16 00:00:00','2019-09-15 00:00:00','2019-10-15 00:00:00','2019-11-14 00:00:00',
            '2019-12-14 00:00:00','2020-01-13 00:00:00','2020-02-12 00:00:00','2020-03-13 00:00:00',
            '2020-04-12 00:00:00','2020-05-12 00:00:00']

len(dates_feat), len(dates_targ)

(10, 10)

In [6]:
%%time

for i in range(len(dates_feat)):
    print("\n*** t=",i,"***")
    merged=pd.merge(real_data.loc[real_data.dt_t0==dates_feat[i],:],targets.loc[targets.dt_t0==dates_targ[i],:], on='ndoc')
    merged=merged.drop(columns=['dt_t0_x','dt_t0_y'])
    merged.to_csv('data/dataset_t'+str(i)+'_complete.csv')


*** t= 0 ***

*** t= 1 ***

*** t= 2 ***

*** t= 3 ***

*** t= 4 ***

*** t= 5 ***

*** t= 6 ***

*** t= 7 ***

*** t= 8 ***

*** t= 9 ***
CPU times: user 51min 24s, sys: 10min 36s, total: 1h 2min 1s
Wall time: 1h 1min 23s
